In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',30)

In [2]:
name = 'Premier Hospital Limited'
df = pd.read_csv('CLIENT_EMAILS_DATA-20240622T091547Z-001/CLIENT_EMAILS_DATA/Premier Hospital Limited.csv')

In [3]:
df = df.drop_duplicates(subset = 'CONTENT')
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
df.drop('Unnamed: 0',axis=1,inplace=True)
df

,SUBJECT,FROM,TO,CONTENT


## Fetching Clients Emails From Emails Threads

In [362]:
import re
j=0
complete_client_emails =[]
complete_emp_mails =[]
complete_emp_mails_to=[]
complete_emp_mails_from=[]
complete_emp_mails_subject =[]
complete_emp_mails_cli_name=[]
To=[]
From=[]
subject=[]
cli_name = []
while j<len(df['CONTENT']):
    text =df['CONTENT'][j]
    # Split the text based on email metadata patterns
    email_pattern = re.compile(r'(\*\*From:\*\*.*?)(?=\*\*From:\*\*|\Z)', re.DOTALL)
    emails = email_pattern.findall(text)
    emails_list=[]
    # Print each email content
    for idx, email in enumerate(emails):
        emails_list.append(email)
        print(f"Email {idx + 1}:\n")
        print(email.strip())
        # print("\n" + "-"*80 + "\n")
    
    i=0
    while i<len(emails_list):
        text = emails_list[i]
        words = re.findall(r'\b\w+\b', text)
        names = f"{words[1]+' '+words[2]}"
        if names =='Carolyn Njange':
           From.append(names)
           complete_client_emails.append(emails_list[i])
           if 'To' in words:
               start = words.index('To')
               To.append(words[start+1]+' '+words[start+3]) 
               cli_name.append(name)
        else:
            complete_emp_mails_from.append(names)
            complete_emp_mails.append(emails_list[i])
            if 'To' in words:
                start = words.index('To')
                complete_emp_mails_to.append(words[start+1]+' '+words[start+3]) 
                complete_emp_mails_cli_name.append(name)
            
        i+=1
    
    j+=1

print(complete_client_emails)

Email 1:

**From:** Joseph Munyao <[jmunyao@imlu.org](mailto:jmunyao@imlu.org)>  
> **Sent:** Monday, October 30, 2023 5:25 PM  
> **To:** [Phoebe.Ochiel@kra.go.ke](mailto:Phoebe.Ochiel@kra.go.ke)
> <[Phoebe.Ochiel@kra.go.ke](mailto:Phoebe.Ochiel@kra.go.ke)>  
> **Cc:** Henry Sang <[henry.sang@bdo-ea.com](mailto:henry.sang@bdo-ea.com)>;
> [Amysera.Ngari@kra.go.ke](mailto:Amysera.Ngari@kra.go.ke)
> <[Amysera.Ngari@kra.go.ke](mailto:Amysera.Ngari@kra.go.ke)>;
> [Elvis.Gichuhi@kra.go.ke](mailto:Elvis.Gichuhi@kra.go.ke)
> <[Elvis.Gichuhi@kra.go.ke](mailto:Elvis.Gichuhi@kra.go.ke)>; Peter Mwangi
> <[peter.mwangi@bdo-ea.com](mailto:peter.mwangi@bdo-ea.com)>; Peter Kariuki
> <[peter.kariuki@bdo-ea.com](mailto:peter.kariuki@bdo-ea.com)>; Peter Kiama
> <[pkiama@imlu.org](mailto:pkiama@imlu.org)>; Charity Mugo
> <[cmugo@imlu.org](mailto:cmugo@imlu.org)>; Stephen Kanyi
> <[skanyi@imlu.org](mailto:skanyi@imlu.org)>; Imlu M&E
> <[smukoya@imlu.org](mailto:smukoya@imlu.org)>; Carolyn Njange
> <[cnjan

## Extracting Mail main content

In [363]:
def Extract_main_content(emails):
    filtered_mails=[]
    unfiltered_mails =[]
    for mail in emails:
        email = mail
            # Pattern to match metadata lines
        metadata_pattern = re.compile(r'\*\*From:.*?\n\*\*Sent:.*?\n\*\*To:.*?\n(\*\*Cc:.*?\n)?\*\*Subject:.*?\n\n', re.DOTALL)
                
            # Remove the metadata lines
        main_content = re.sub(metadata_pattern, '', email).strip()
                
                
        text = main_content
        # print(text)
        words = re.findall(r'\b\w+\b', text)
            # Iterate through the list of words
        sent = ''
        ending_lis = ['Regards','Rgds','DISCLAIMER']
        for item in ending_lis:
            if item in words:
                index = words.index(item)
                for word in words[:index-2]:
                    sent = sent+" "+word
                # print(sent,'\n\n')
                filtered_mails.append(sent)
            
        sent = ''
        for word in words:
             sent = sent+" "+word 
        unfiltered_mails.append(sent) 
    return unfiltered_mails


unfiltered_mails_client = Extract_main_content(complete_client_emails)
unfiltered_mails_emp = Extract_main_content(complete_emp_mails)

## Extracting mail Subject 

In [364]:
def fetching_subject(emails,greeting_words):
    j=0
    end =1000
    subject=[]
    while j<len(emails):
        text = emails[j]
        if len(text)!=0:
            words = re.findall(r'\b\w+\b', text)
            
            # Iterate through the list of words
            sent = ''
            if 'Subject'  in words:
                start = words.index('Subject')
            for greeting in greeting_words:
                if greeting in words:
                    ending = words.index(greeting)
                    if ending-start<end and ending-start>3 :
                        end=ending-start
                    # break
            # print(final)
            # print(start)
            # print(end)
            for word in words[start+1:start+end]:
                sent = sent+" "+word
            subject.append(sent)
        j+=1
    return subject


def find_greeting_word(mails):
    i=0
    greetings =[]
    while i<len(mails):
        text = mails[i]
        if len(text)!=0:
            first_word = text.split()[0]
            greetings.append(first_word)
        i+=1
    greeting_words = list(set(greetings))
    return greeting_words

greeting_emp = find_greeting_word(unfiltered_mails_emp)
greeting_client = find_greeting_word(unfiltered_mails_emp)
subject_client = fetching_subject(complete_client_emails,greeting_client)
subject_emp = fetching_subject(complete_emp_mails,greeting_emp)

In [365]:
client_emails={
    'cli_name':cli_name,
    'Subject':subject_client,
    'From':From,
    'To':To,
    'Text':unfiltered_mails_client
}
emp_mails={
    'emp_name':complete_emp_mails_cli_name,
    'Subject':subject_emp,
    'From':complete_emp_mails_from,
    'To':complete_emp_mails_to,
    'Text':unfiltered_mails_emp
}

In [366]:
df1 = pd.DataFrame(client_emails)
df2 = pd.DataFrame(emp_mails)

In [367]:
df1 = df1.drop_duplicates(subset='Text')
df2 = df2.drop_duplicates(subset='Text')

In [368]:
df1.reset_index(inplace=True)
df1.drop('index',axis=1,inplace=True)
df2.reset_index(inplace=True)
df2.drop('index',axis=1,inplace=True)

In [369]:
df1

,cli_name,Subject,From,To,Text
0,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for Income Tax Exemption Certi...,Carolyn Njange,Peter peter,Dear Mwangi This is fine with us Regards On T...
1,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for Income Tax Exemption Certi...,Carolyn Njange,Phoebe kra,From Carolyn Njange cnjange imlu org mailto c...
2,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for Income Tax Exemption Certi...,Carolyn Njange,Phoebe kra,Dear Mwangi We spoke when Phoebe requested th...
3,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for Income Tax Exemption Certi...,Carolyn Njange,Peter peter,Dear Kariuki Thank you for that clarification...
4,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Fwd Application for Income Tax Exemption Cert...,Carolyn Njange,Peter peter,From Carolyn Njange cnjange imlu org mailto c...
5,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Fwd Application for Income Tax Exemption Cert...,Carolyn Njange,Peter peter,Dear BDO Team See list of requirements from K...
6,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for the Renewal of Income Tax ...,Carolyn Njange,Peter peter,Dear Peter Find attached the filled tax payer...
7,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re URGENT Update of compliance audit progress,Carolyn Njange,Peter peter,Dear Mwangi Kindly revert to us as promised e...
8,INDEPENDENT MEDICO LEGAL UNIT (IMLU),URGENT Update of compliance audit progress,Carolyn Njange,Peter peter,From Carolyn Njange cnjange imlu org mailto c...
9,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for the Renewal of,Carolyn Njange,Peter peter,Dear Mwangi Find attached the TCC for our Boa...


In [286]:
df2[df2['From']=='Shiraz Rehmanji']

,emp_name,Subject,From,To,Text


In [370]:
df2

,emp_name,Subject,From,To,Text
0,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for Income Tax Exemption Certi...,Joseph Munyao,Phoebe kra,From Joseph Munyao jmunyao imlu org mailto jm...
1,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for Income Tax Exemption Certi...,Joseph Munyao,Phoebe kra,From Joseph Munyao jmunyao imlu org mailto jm...
2,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for Income Tax Exemption Certi...,Joseph Munyao,Phoebe kra,From Joseph Munyao jmunyao imlu org mailto jm...
3,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Application for Income Tax Exemption Certi...,Joseph Munyao,Phoebe kra,From Joseph Munyao jmunyao imlu org mailto jm...
4,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Statement of Account with BDO Kenya Limite...,Charity Mugo,Carolyn cnjange,Dear Daniel Kindly share with us the invoice ...
5,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re Statement of Account with BDO Kenya Limite...,Peter Mwangi,George george,Dear George Yes you are right One is for tax ...
6,INDEPENDENT MEDICO LEGAL UNIT (IMLU),FW Statement of Account with BDO Kenya Limite...,George Ndungu,Peter peter,Peter Mwangi mailto peter mwangi bdo ea com J...
7,INDEPENDENT MEDICO LEGAL UNIT (IMLU),FW Statement of Account with BDO Kenya Limite...,Abdul Zahir,George george,Please chase this
8,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Statement of Account with BDO Kenya Limited a...,Daniel Mulwa,cnjange org,Dear Caroline Njange I hope this message find...
9,INDEPENDENT MEDICO LEGAL UNIT (IMLU),Re URGENT Update of compliance audit progress,Peter Mwangi,Carolyn cnjange,Dear Carolyn Trust all is well Please find at...


In [371]:
df1.to_csv(f'Client/{name}.csv')
df2.to_csv(f'Emp/{name}.csv')

In [57]:
j=0
end =1000
subject=[]
while j<len(complete_client_emails):
    text = complete_client_emails[j]
    words = re.findall(r'\b\w+\b', text)
    
    # Iterate through the list of words
    sent = ''
    if 'Subject'  in words:
        start = words.index('Subject')
    for greeting in greeting_words:
        if greeting in words:
            ending = words.index(greeting)
            if ending-start<end and ending-start>3 :
                end=ending-start
            # break
    # print(final)
    print(start)
    print(end)
    for word in words[start+1:start+end]:
        sent = sent+" "+word
    subject.append(sent)
    j+=1

54
7
89
7
80
7
76
7
70
6
80
6
42
5
34
5
24
4
54
4
80
4
76
4
70
4
80
4
42
4
34
4
24
4
49
4
76
4
70
4
80
4
42
4
34
4
24
4
56
4
92
4
79
4
56
4
79
4
56
4
79
4
49
4
28
4
42
4
31
4
28
4
21
4
28
4
21
4
47
4
44
4
80
4
42
4
34
4
24
4
22
4
42
4
22
4
32
4
22
4
45
4
22
4
75
4
73
4
115
4
61
4
41
4
41
4
23
4
22
4
54
4
32
4
22
4
35
4
56
4
55
4
42
4
35
4
56
4
55
4
42
4
35
4
56
4
55
4
42
4
36
4
55
4
42
4
41
4
41
4
27
4
49
4
42
4
42
4
46
4
55
4
55
4
46
4
47
4
55
4
48
4
55
4
55
4
46
4
47
4
55
4
35
4
55
4
46
4
47
4
55
4
35
4
46
4
47
4
55
4
42
4
42
4
35
4
27
4
35
4
27
4
22
4
22
4
36
4
55
4
42
4
35
4
42
4
28
4
28
4
22
4
33
4
23
4
33
4
23
4
31
4
46
4
30
4
35
4
32
4
55
4
42
4
30
4
35
4
42
4
28
4
30
4
41
4
27
4
22
4
19
4
35
4
42
4
35
4
22
4
19
4
35
4
42
4
35
4
28
4
48
4
78
4
79
4
78
4
78
4
80
4
42
4
34
4
24
4
48
4
79
4
78
4
78
4
80
4
42
4
34
4
24
4
49
4
78
4
78
4
80
4
42
4
34
4
24
4
48
4
78
4
80
4
42
4
34
4
24
4
48
4
78
4
80
4
42
4
34
4
24
4
49
4
80
4
42
4
34
4
24
4
48
4
80
4
42
4
34
4
24
4
22
4
78
4
78
4
78
4

In [54]:
complete_client_emails

['**From:** Ernest Njuguna <enjuguna@peponischool.org>  \n**Sent:** Thursday, June 13, 2024 11:24 AM  \n**To:** Henry Sang <henry.sang@bdo-ea.com>  \n**Cc:** Human Resource <HR@peponihouse.sc.ke>; Timothy Mwangi\n<tmwangi@peponischool.org>; Peter Kariuki <peter.kariuki@bdo-ea.com>; Michael\nMutunga (Staff) <m.mutunga@peponihouse.sc.ke>; Geoffrey Wambua\n<gwambua@peponischool.org>  \n**Subject:** RE: SOL Tax Reconciliations with KRA\n\n\n\nPlease appreciate that the TCC has expired.\n\n\n\ne\n\n\n\nErnest Njuguna,  \nBursar  \n  \n\n  \n\n* * *\n\nDISCLAIMER: The information contained in this e-mail and any attachments\nthereto is privileged and confidential. If you are not the intended recipient\nany review, dissemination or use of this transmission or its contents is\nstrictly prohibited. Kindly notify the sender promptly via return email and\ndelete this message and its attachments immediately. The Institution and\nsender accept no liability whatsoever for any loss or damages, howeve

In [161]:
emails_list[0]

'**From:** Richard Kasibante <richard.kasibante@gizasystems.com>  \n**Sent:** Wednesday, 19 June 2024 11:39  \n**To:** Nanyonjo Ritah Jacinta <ritah.nanyonjo@bdo-ea.com>  \n**Cc:** Ehab George <ehab.george@gizasystems.com>; Ahmed Hany\n<ahmed.hany@gizasystems.com>  \n**Subject:** RE: Rejection of TCC\n\n\n\nDear Ritah,\n\n\n\nAs per our phone call, please find below the details of Ahmed\n\n\n\n  1. TIN 1016084617\n  2. Email address: [ahmed.hany@gizasystems.com](mailto:ahmed.hany@gizasystems.com)\n\n\n\nRegards,\n\n\n\nRichard\n\n\n\n'

In [7]:
i=0
client_emails=[]
while i<len(emails_list):
    text = emails_list[i]
    words = re.findall(r'\b\w+\b', text)
    names = f"{words[1]+' '+words[2]}"
    if names=='Ernest Njuguna':
# # Iterate through the list of words
       client_emails.append(emails_list[i])
    i+=1

In [8]:
client_emails

[]

In [45]:
def extract_main_content(email_text):
    # Define patterns to remove signatures and unnecessary parts
    signature_pattern = r"\*\*From:\*\*.*|\*\*Sent:\*\*.*|\*\*To:\*\*.*|\*\*Cc:\*\*.*|\*\*Subject:\*\*.*|\*\*.*?\*\*|<[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}>|_[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}_|\[[^\]]*\]\([^)]*\)"

    # Remove signatures, URLs, and unnecessary text
    filtered_text = re.sub(signature_pattern, "", email_text)

    # Trim leading and trailing whitespace
    filtered_text = filtered_text.strip()

    return filtered_text


filtered_content = extract_main_content(main_content)
print(filtered_content)

<>;
Ahmed Hany <>;
Salim Sempera
<>  

<>  




Hi Richard,



Let me prepare the relevant documentation i.e., Resolution and Form 20 and
share for your review.



Kind Regards,



  
  
+256 752 644 461

+256 771 014 392 (WhatsApp only)  
  __ __


In [ ]:
while i<len(emails_list):
    text = emails_list[i]
    words = re.findall(r'\b\w+\b', text)
            # Iterate through the list of words
    sent = ''
    print(words[1]+words[2])
    # if (words[1]+' '+words[2])=='Ahmed Hany':
    #     for word in words:
    #         sent = sent+" "+word
    #     print(sent,'\n\n')

RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasibante
RichardKasiban

In [9]:
import re
complete_fileterd_mails =[]
for mails in list(df['CONTENT']):
    # print(mails)
    # break
    text = mails 
    # Split the text based on email metadata patterns
    email_pattern = re.compile(r'(\*\*From:\*\*.*?)(?=\*\*From:\*\*|\Z)', re.DOTALL)
    emails = email_pattern.findall(text)
    emails_list=[]
     # Print each email content
    for idx, email in enumerate(emails):
        emails_list.append(email)
        # print(f"Email {idx + 1}:\n")
        # print(email.strip())
        # print("\n" + "-"*80 + "\n")
        
    # print(emails_list)
    filtered_mails = []
    for mail in emails_list:
        email = mail
        # Pattern to match metadata lines
        metadata_pattern = re.compile(r'\*\*From:.*?\n\*\*Sent:.*?\n\*\*To:.*?\n(\*\*Cc:.*?\n)?\*\*Subject:.*?\n\n', re.DOTALL)
            
        # Remove the metadata lines
        main_content = re.sub(metadata_pattern, '', email).strip()
            
            
        text = main_content
        words = re.findall(r'\b\w+\b', text)
        # Iterate through the list of words
        sent = ''
        for word in words:
            sent = sent+" "+word
        print(sent,'\n\n')
        filtered_mails.append(sent)
    complete_fileterd_mails.append(filtered_mails)

 Please appreciate that the TCC has expired e Ernest Njuguna Bursar DISCLAIMER The information contained in this e mail and any attachments thereto is privileged and confidential If you are not the intended recipient any review dissemination or use of this transmission or its contents is strictly prohibited Kindly notify the sender promptly via return email and delete this message and its attachments immediately The Institution and sender accept no liability whatsoever for any loss or damages however incurred or suffered resulting or arising from the use of this email or its attachments 


 Hello Ernest Thanks for your email i will get back to you on this Regards Henry Sang Director Business Tax Advisory Services Mobile 254 722 654455 Direct line 254 709 254 712 henry sang bdo ea com mailto henry sang bdo ea com Contact address BDO East Africa Kenya The Westwood 9th Floor Vale Close Off Ring Road Westlands P O BOX 10032 00100 Nairobi Kenya www bdo ea com https www bdo ea com en gb bdo 

In [10]:
filtered_mails=[]
unfiltered_mails =[]
for mail in complete_client_emails:
    email = mail
        # Pattern to match metadata lines
    metadata_pattern = re.compile(r'\*\*From:.*?\n\*\*Sent:.*?\n\*\*To:.*?\n(\*\*Cc:.*?\n)?\*\*Subject:.*?\n\n', re.DOTALL)
            
        # Remove the metadata lines
    main_content = re.sub(metadata_pattern, '', email).strip()
            
            
    text = main_content
    # print(text)
    words = re.findall(r'\b\w+\b', text)
        # Iterate through the list of words
    sent = ''
    ending_lis = ['Regards','Rgds','DISCLAIMER']
    for item in ending_lis:
        if item in words:
            index = words.index(item)
            for word in words[:index-2]:
                sent = sent+" "+word
            # print(sent,'\n\n')
            filtered_mails.append(sent)
        
    sent = ''
    for word in words:
         sent = sent+" "+word 
    unfiltered_mails.append(sent)          

In [11]:
len(filtered_mails),len(complete_client_emails),len(unfiltered_mails)

(219, 234, 234)

In [12]:
unfiltered_mails[0]

' Please appreciate that the TCC has expired e Ernest Njuguna Bursar DISCLAIMER The information contained in this e mail and any attachments thereto is privileged and confidential If you are not the intended recipient any review dissemination or use of this transmission or its contents is strictly prohibited Kindly notify the sender promptly via return email and delete this message and its attachments immediately The Institution and sender accept no liability whatsoever for any loss or damages however incurred or suffered resulting or arising from the use of this email or its attachments'

In [15]:
def find_greeting_word(mails):
    i=0
    greetings =[]
    while i<len(unfiltered_mails):
        text = unfiltered_mails[i]
        first_word = text.split()[0]
        greetings.append(first_word)
        i+=1
    greeting_words = (list(set(greetings)))
    return greeting_words

In [27]:
greeting_words[4]

'MG'

In [28]:
greeting_words

['Ernest',
 'Thank',
 'Good',
 'A',
 'MG',
 'Noted',
 'Now',
 'No',
 'TMM',
 'MW',
 'Kindly',
 'Please',
 'Agreed',
 'Awaiting',
 'WARNING',
 'Dear',
 'MM',
 'Thanks',
 'Not']

In [355]:
complete_emp_mails_to=[]
complete_emp_mails_from=[]
subject_emp =[]
complete_emp_mails_cli_name=[]
unfiltered_mails_emp=[]
name = 'Equatorial Nut Processors Limited'
i=0
while i<len(df['CONTENT']):
    text = df['CONTENT'][i]
    words = re.findall(r'\b\w+\b', text)
            # Iterate through the list of words
    sent = ''
    if 'Sincerely' in words:
        end = words.index('Sincerely')
    for word in words[:end]:
        sent = sent+" "+word 
    complete_emp_mails_cli_name.append(name)
    unfiltered_mails_emp.append(sent)
    complete_emp_mails_from.append(words[end+1]+' '+words[end+2])
    complete_emp_mails_to.append(words[1]+' '+words[2]+' '+words[3])  
    subject_emp.append(df['SUBJECT'][i])
    i+=1

In [356]:
emp_mails={
    'emp_name':complete_emp_mails_cli_name,
    'Subject':subject_emp,
    'From':complete_emp_mails_from,
    'To':complete_emp_mails_to,
    'Text':unfiltered_mails_emp
}

In [357]:
df2 = pd.DataFrame(emp_mails)

In [358]:
df2

,emp_name,Subject,From,To,Text
0,Equatorial Nut Processors Limited,Statement of Account with BDO Kenya Limited as...,Daniel Mulwa,Dr Moses Mwangi,Dear Dr Moses Mwangi I hope this message find...
1,Equatorial Nut Processors Limited,Statement of Account with BDO Kenya Limited as...,Daniel Mulwa,Dr Moses Mwangi,Dear Dr Moses Mwangi I hope this message find...
2,Equatorial Nut Processors Limited,Statement of Account with BDO Kenya Limited as...,Daniel Mulwa,Dr Moses Mwangi,Dear Dr Moses Mwangi I hope this message find...
3,Equatorial Nut Processors Limited,Statement of Account with BDO Kenya Limited as...,Daniel Mulwa,Dr Moses Mwangi,Dear Dr Moses Mwangi I hope this message find...
4,Equatorial Nut Processors Limited,Statement of Account with BDO Kenya Limited as...,Daniel Mulwa,Dr Moses Mwangi,Dear Dr Moses Mwangi I hope this message find...
5,Equatorial Nut Processors Limited,Statement of Account with BDO Kenya Limited as...,Daniel Mulwa,Dr Moses Mwangi,Dear Dr Moses Mwangi I hope this message find...
6,Equatorial Nut Processors Limited,Statement of Account with BDO Kenya Limited as...,Daniel Mulwa,Dr Moses Mwangi,Dear Dr Moses Mwangi I hope this message find...
